# NextValue

## Bibliotecas Necessárias

In [1]:
import pandas as pd
from os import listdir
from os.path import isfile, join
import numpy as np
import math
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

## Processamento dos Dados

### Importando o Dataset

In [2]:
DATA_PATH = "./Prices"

dataset_list = [f for f in listdir(DATA_PATH) if (isfile(join(DATA_PATH, f)) and f.endswith(".csv"))]
dataset_list.sort()


### Dados Necessários

In [3]:
NUMBER_OF_CRYPTOCURRENCIES = len(dataset_list)

cryptocurrency_list = [d[:-4] for d in dataset_list]
index = [i for i in range(NUMBER_OF_CRYPTOCURRENCIES)]

CRYPTOCURRENCY_INDEX = dict(zip(cryptocurrency_list, index))

### Jutando os datasets

In [4]:
dataset = pd.read_csv(DATA_PATH + "/" + dataset_list[0])
dataset = dataset.iloc[:, 1:3]

for i in range(1, NUMBER_OF_CRYPTOCURRENCIES):
    file = pd.read_csv(DATA_PATH + "/" + dataset_list[i])
    file = file.iloc[:, 1:3]
    dataset = pd.merge(dataset, file, on='Date', how='outer')

dataset = dataset.iloc[:, 1:NUMBER_OF_CRYPTOCURRENCIES+1]
dataset = dataset.replace(np.nan, 0)

### Dividindo o Dataset em TrainingSet e TestSet

In [5]:
NUMBER_OF_ROWS = dataset.shape[0]
TRAINING_RATE = 0.95
TRAINING_SET_SIZE = math.ceil(NUMBER_OF_ROWS*TRAINING_RATE)
TEST_SET_SIZE = NUMBER_OF_ROWS - TRAINING_SET_SIZE

training_set = dataset.iloc[:TRAINING_SET_SIZE, : ].values
test_set = dataset.iloc[TRAINING_SET_SIZE:, : ].values

### Feature Scaling

In [6]:
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)

### Construindo a Estrutura de Dados

In [7]:
DIAS_ANTERIORES = 30

X_train = []
y_train = []
for i in range(DIAS_ANTERIORES, TRAINING_SET_SIZE):
    X_train.append(training_set_scaled[i-DIAS_ANTERIORES:i, 0:NUMBER_OF_CRYPTOCURRENCIES])
    y_train.append(training_set_scaled[i, 0:NUMBER_OF_CRYPTOCURRENCIES])

X_train, y_train = np.array(X_train), np.array(y_train)

### Reshaping

In [8]:
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], NUMBER_OF_CRYPTOCURRENCIES))

## Construindo a Rede Neural

### Inicializando a Rede

In [9]:
UNITS = 500

regressor = Sequential()

### Adicionando a primeira camada LSTM e Dropout

In [10]:
regressor.add(LSTM(units = UNITS, return_sequences = True, input_shape = (X_train.shape[1], NUMBER_OF_CRYPTOCURRENCIES)))
regressor.add(Dropout(0.2))

### Adicionando mais 1 camada LSTM e Dropout

In [11]:
regressor.add(LSTM(units = UNITS, return_sequences = True))
regressor.add(Dropout(0.2))

### Adicionando a última camada LSTM e Dropout

In [12]:
regressor.add(LSTM(units = UNITS))
regressor.add(Dropout(0.2))

### Adicionando a camada de saída

In [13]:
regressor.add(Dense(units = NUMBER_OF_CRYPTOCURRENCIES))

### Compilando a Rede

In [14]:
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

## Treinando a Rede Neural

In [ ]:
regressor.fit(X_train, y_train, epochs = 100, batch_size = 128)

## Testando a Rede Neural

In [ ]:
inputs = dataset[TRAINING_SET_SIZE-DIAS_ANTERIORES:]
inputs = inputs.reshape((inputs.shape[0], inputs.shape[1]))
inputs = sc.transform(inputs)

X_test = []
for i in range(DIAS_ANTERIORES, DIAS_ANTERIORES+TEST_SET_SIZE):
  X_test.append(inputs[i-DIAS_ANTERIORES:i, 0:NUMBER_OF_CRYPTOCURRENCIES])

X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], NUMBER_OF_CRYPTOCURRENCIES))

predict = regressor.predict(X_test)
predict = sc.inverse_transform(predict)

## Visualizando os Resultados

In [ ]:
CRYPTOCURRENCY = "Bitcoin"

INDEX_OF_CRYPTOCURRENCY = CRYPTOCURRENCY_INDEX[CRYPTOCURRENCY]

test_cryptocurrency = []
predict_cryptocurrency = []

for i in range(TEST_SET_SIZE):
  test_cryptocurrency.append(test_set[i, INDEX_OF_CRYPTOCURRENCY])
  predict_cryptocurrency.append(predict[i, INDEX_OF_CRYPTOCURRENCY])

plt.plot(test_cryptocurrency, color = 'red', label = 'Real ' + CRYPTOCURRENCY + ' Price')
plt.plot(predict_cryptocurrency, color = 'blue', label = 'Predicted ' + CRYPTOCURRENCY + ' Price')
plt.title(CRYPTOCURRENCY + ' Price Prediction')
plt.xlabel('Time')
plt.ylabel(CRYPTOCURRENCY + ' Price')
plt.legend()
plt.show()